In [ ]:
# Code for CNN Based Method

In [ ]:
!pip install opencv-python

In [ ]:
import time
import cv2
from cv2  import dnn_superres
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

image_path='/content/images'
output_path='/content/outputs'


MODEL_LIST = [
    {"name": "EDSR", "path":"/content/EDSR_x2.pb", "scale":2},
    {"name": "ESPCN", "path":"/content/ESPCN_x2.pb", "scale":2},
    {"name": "FSRCNN", "path":"/content/FSRCNN_x2.pb", "scale":2},
    {"name": "LAPSRN", "path":"/content/LapSRN_x2.pb", "scale":2},

    {"name": "EDSR", "path":"/content/EDSR_x3.pb", "scale":3},
    {"name": "ESPCN", "path":"/content/ESPCN_x3.pb", "scale":3},
    {"name": "FSRCNN", "path":"/content/FSRCNN_x3.pb", "scale":3},

    {"name": "EDSR", "path":"/content/EDSR_x4.pb", "scale":4},
    {"name": "ESPCN", "path":"/content/ESPCN_x4.pb", "scale":4},
    {"name": "FSRCNN", "path":"/content/FSRCNN_x4.pb", "scale":4},
    {"name": "LAPSRN", "path":"/content/LapSRN_x4.pb", "scale":4},

    {"name": "LAPSRN", "path":"/content/LapSRN_x8.pb", "scale":8},
]

def calculate_psnr(original,new):
    return cv2.PSNR(original,new)

def calculate_mse(original,new):
    return (np.square(original - new)).mean(axis=None)

def calculate_ssim(original,new):
    return ssim(original, new, channel_axis=2)

def save_image(path, result):
    cv2.imwrite(path,result)

def model_run(enable_write):
    for model in MODEL_LIST:
        name = model['name']
        path = model['path']
        scale = model['scale']

        if name!='interpolation':
            sr=dnn_superres.DnnSuperResImpl_create()
            sr.readModel(path)
            sr.setModel(name.lower(),scale)

        for i in os.listdir(image_path):
            input_image=cv2.imread(os.path.join(image_path,i))
            input_image = cv2.resize(input_image, (int(input_image.shape[1]/scale),int(input_image.shape[0]/scale)))
            processed_image = cv2.resize(input_image, (int(input_image.shape[1]*scale),int(input_image.shape[0]*scale)))

            path=os.path.join(output_path,f'x{scale}')
            path=os.path.join(path,f'{name}_x{scale}.pb')
            path=os.path.join(path,i)

            begin=time.time()
            result_image=sr.upsample(input_image)
            interval=time.time()-begin
            print(scale)
            print(name)
            print(f'Time is : {interval}')
            print(f'PSNR is : {calculate_psnr(processed_image, result_image)}')
            print(f'MSE is : {calculate_mse(processed_image, result_image)}')
            print(f'SSIM is : {calculate_ssim(processed_image, result_image)}')

            print('')
            if enable_write==True:
                save_image(path,result_image)

model_run(True)

# Code for ESRGAN

In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

In [ ]:
import torch
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN
import time
from skimage.metrics import mean_squared_error, structural_similarity

# Setting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

# Path to the input image
path_to_image = 'Your image path'
image = Image.open(path_to_image).convert('RGB')

# Start timing
start_time = time.time()

# Run the model
sr_image = model.predict(image)

# End timing
end_time = time.time()

# Save the output image
sr_image.save('Your output image path')

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to run the RealESRGAN model: {elapsed_time:.2f} seconds")

# Convert images to numpy arrays for metric calculations
input_image_np = np.array(image)
sr_image_np = np.array(sr_image)

# Ensure both images have the same dimensions for comparison
if input_image_np.shape != sr_image_np.shape:
    sr_image_np = np.array(sr_image.resize(image.size))

# Calculate MSE
mse_score = mean_squared_error(input_image_np, sr_image_np)
print(f"MSE score: {mse_score}")

# Calculate SSIM
ssim_score = structural_similarity(input_image_np, sr_image_np, multichannel=True)
print(f"SSIM score: {ssim_score}")